In [1]:
##########################################

# Start here once given anonymized csvs.

###########################################
# %matplotlib notebook
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D
import os
import LLB_custom_scripts_mac
from LLB_custom_scripts_mac import make_module_summary
import csv
from collections import Counter
import re

In [2]:
# Loop through anonymized files and save the dataframes 
# of progression and module files in separate dictionaries.
# Also make a dictionary with the number of assessments in 
# each module, each year.

# Paths for locating anonymized csvs:
anon_module_csv_path = ("/Users/Kate/Desktop/Vicky project/"
                        "Data/anonymized module csvs/")

anon_progression_csv_path = ("/Users/Kate/Desktop/Vicky project/"
                            "Data/anonymized progression csvs/")

progression_files = {}
module_files = {}
n_assessments_module_dict = {}

# Loop
for filename in os.listdir(anon_module_csv_path):
    if filename == '.DS_Store': 
        continue
    else:
        # Convert csv to dataframe and store in a dictionary.
        module_name = f'{filename}'.split('.')[0]
        module_files[module_name] = pd.read_csv(
            anon_module_csv_path + f'{filename}')
        module_files[module_name].set_index('SPR Code', inplace=True)
        num_assessments = module_files[module_name].columns[-1].split(' ')[1]
        n_assessments_module_dict[module_name] = num_assessments
for filename in os.listdir(anon_progression_csv_path):
    if filename == '.DS_Store': 
        continue
    else:
        # Convert csv to dataframe and store in a dictionary.
        progression_files[f'{filename}'.split('.')[0]] = pd.read_csv(
            anon_progression_csv_path + f'{filename}')
        progression_files[f'{filename}'.split('.')[0]].set_index(
            'SPR Code', inplace=True)

In [3]:
# Store the 201011 module files for comparison
anon_module_path_200910 = ("/Users/Kate/Desktop/Vicky project/"
                        "Data/anonymized module csvs - starting 2009-2010/")

mod_files_2010 ={}
for filename in os.listdir(anon_module_path_200910):
    if filename == '.DS_Store':
        continue
    else:
        year, module, tmp = f'{filename}'.split('.')[0].split('_')
        if year == '201011':
            df = pd.read_csv(anon_module_path_200910 + f'{filename}')
            mod_files_2010[module] = df.set_index('SPR Code')
                                                 

In [4]:
# Remove all students with blank results
for dfname, df in module_files.items():
    module_files[dfname] = df[df['Result'].notnull()]

In [5]:
# Remove all students not in the LLB programme
for dfname, df in module_files.items():
    module_files[dfname] = df[df['Programme'].str.startswith('LLB Law')] 

In [6]:
# Remove all duplicated student entries from module dataframes 
# since confusion about a variable, will have to remove all that were duplicated
for dfname, df in module_files.items():
    module_files[dfname] = df[~df.index.duplicated(keep=False)]

In [7]:
# Add reassess flag
def determine_reassess_and_dr_flags(df):
    df['Reassess Flag'] = df.apply(lambda row: any(row.isin(['FR'])), axis = 1)   
    df['DR Flag'] = df.apply(lambda row: (any(row.isin(['DR'])) & (row['Result'] != 'D')), axis = 1) 
    ######### better way to catch assessment DRs only??? ########
    return df
            
for dfname, df in module_files.items():
    module_files[dfname] = determine_reassess_and_dr_flags(df)

In [8]:
# Create student records dictionary

# create dataframes with 
student_attempts = pd.DataFrame()
student_grades = pd.DataFrame()
student_reassess_flags = pd.DataFrame()
student_dr_flags = pd.DataFrame()
student_marks = pd.DataFrame()

for dfname, df in module_files.items():
    year, module, tmp = dfname.split('_')
    student_attempts = student_attempts.join(df[['Attempt']], how='outer')
    student_attempts.rename(columns={'Attempt': f'{year} {module}'}, inplace=True)
    student_grades = student_grades.join(df[['Grade']], how='outer')
    student_grades.rename(columns={'Grade': f'{year} {module}'},inplace=True)
    if any(df.columns == 'Mark'):
        student_marks = student_marks.join(df[['Mark']], how='outer')
        student_marks.rename(columns={'Mark':f'{year} {module}'}, inplace=True)
    else:
        student_marks = student_marks.join(df[['Grade']], how='outer')
        student_marks.rename(columns={'Grade':f'{year} {module}'}, inplace=True)
        student_marks[f'{year} {module}'] = np.NaN
    student_reassess_flags = student_reassess_flags.join(df[['Reassess Flag']], how='outer')
    student_reassess_flags.rename(columns={'Reassess Flag': f'{year} {module}'},inplace=True)
    student_dr_flags = student_dr_flags.join(df[['DR Flag']], how='outer')
    student_dr_flags.rename(columns={'DR Flag': f'{year} {module}'},inplace=True)
    
student_attempts = student_attempts.reindex(sorted(student_attempts.columns), axis=1)#sort by year

In [9]:
student_zipped = pd.DataFrame(index = student_attempts.index, columns = student_attempts.columns)
for index in student_attempts.index:
    for column in student_attempts.columns:
        if student_marks.at[index, column] == np.NaN:
            student_zipped.at[index, column] = str(student_reassess_flags.at[index, column]) \
                                            + ' ' + str(student_dr_flags.at[index, column]) \
                                            + ' ' + str(student_attempts.at[index,column]) \
                                            + ' ' + 'nan' + ' ' + str(student_grades.at[index,column])
        else:
            student_zipped.at[index, column] = str(student_reassess_flags.at[index, column]) \
                                            + ' ' + str(student_dr_flags.at[index, column]) \
                                            + ' ' + str(student_attempts.at[index,column]) \
                                            + ' ' + str(student_marks.at[index,column]) \
                                            + ' ' + str(student_grades.at[index,column])

In [10]:
student_zipped.head()

,201112 CONAD,201112 CONTRACT,201112 CRIMINAL,201112 EQUITY,201112 EU,201112 LAND,201112 LSM,201112 LT1,201112 LT2,201112 TORT,...,201718 CONAD,201718 CONTRACT,201718 CRIMINAL,201718 EQUITY,201718 EU,201718 LAND,201718 LSM,201718 LT1,201718 LT2,201718 TORT
SPR Code,,,,,,,,,,,,,,,,,,,,,
00016413-5863-4441-af2c-9743f19bf6cd,nan nan nan nan nan,nan nan nan nan nan,nan nan nan nan nan,nan nan nan nan nan,nan nan nan nan nan,nan nan nan nan nan,nan nan nan nan nan,nan nan nan nan nan,nan nan nan nan nan,nan nan nan nan nan,...,False False 1.0 51.0 P,True False 2.0 46.0 P,nan nan nan nan nan,nan nan nan nan nan,nan nan nan nan nan,nan nan nan nan nan,True False 2.0 nan F,nan nan nan nan nan,nan nan nan nan nan,nan nan nan nan nan
001e2bba-90fa-4a62-bf39-ddb29a956d93,nan nan nan nan nan,nan nan nan nan nan,nan nan nan nan nan,nan nan nan nan nan,nan nan nan nan nan,nan nan nan nan nan,nan nan nan nan nan,nan nan nan nan nan,nan nan nan nan nan,nan nan nan nan nan,...,nan nan nan nan nan,nan nan nan nan nan,nan nan nan nan nan,nan nan nan nan nan,nan nan nan nan nan,nan nan nan nan nan,nan nan nan nan nan,nan nan nan nan nan,nan nan nan nan nan,nan nan nan nan nan
0033fc70-2eb9-45f0-990e-d514bab75508,True False 3.0 0.0 W,True False 3.0 22.0 W,nan nan nan nan nan,nan nan nan nan nan,nan nan nan nan nan,nan nan nan nan nan,True False 3.0 12.0 W,nan nan nan nan nan,nan nan nan nan nan,nan nan nan nan nan,...,nan nan nan nan nan,nan nan nan nan nan,nan nan nan nan nan,nan nan nan nan nan,nan nan nan nan nan,nan nan nan nan nan,nan nan nan nan nan,nan nan nan nan nan,nan nan nan nan nan,nan nan nan nan nan
00427b7c-fa09-430f-b505-4b04ad84f055,nan nan nan nan nan,nan nan nan nan nan,False False 1.0 69.0 P,nan nan nan nan nan,nan nan nan nan nan,nan nan nan nan nan,nan nan nan nan nan,nan nan nan nan nan,nan nan nan nan nan,False False 1.0 65.0 P,...,nan nan nan nan nan,nan nan nan nan nan,nan nan nan nan nan,nan nan nan nan nan,nan nan nan nan nan,nan nan nan nan nan,nan nan nan nan nan,nan nan nan nan nan,nan nan nan nan nan,nan nan nan nan nan
00458cb0-d9ce-4520-82d4-f14bc2a7a422,False False 1.0 45.0 P,True False 3.0 39.0 W,nan nan nan nan nan,nan nan nan nan nan,nan nan nan nan nan,nan nan nan nan nan,False False 1.0 47.0 P,nan nan nan nan nan,nan nan nan nan nan,nan nan nan nan nan,...,nan nan nan nan nan,nan nan nan nan nan,nan nan nan nan nan,nan nan nan nan nan,nan nan nan nan nan,nan nan nan nan nan,nan nan nan nan nan,nan nan nan nan nan,nan nan nan nan nan,nan nan nan nan nan


In [11]:
student_zipped = student_zipped.replace('nan nan nan nan nan', np.NaN)

records = {}

for index, row in student_zipped.iterrows():
    courses_on_record = row.dropna().index.tolist()
    results = row.dropna().values.tolist()
    records[index] = list(zip(courses_on_record,results))

RECORDS = {}
for SPRcode, record in records.items():
    new_list = []
    for (course, result) in record:
        new_list.append(course + ' ' + result)
    RECORDS[SPRcode] = new_list
    
len(RECORDS)

1861

In [12]:
student_zipped.head()

,201112 CONAD,201112 CONTRACT,201112 CRIMINAL,201112 EQUITY,201112 EU,201112 LAND,201112 LSM,201112 LT1,201112 LT2,201112 TORT,...,201718 CONAD,201718 CONTRACT,201718 CRIMINAL,201718 EQUITY,201718 EU,201718 LAND,201718 LSM,201718 LT1,201718 LT2,201718 TORT
SPR Code,,,,,,,,,,,,,,,,,,,,,
00016413-5863-4441-af2c-9743f19bf6cd,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,False False 1.0 51.0 P,True False 2.0 46.0 P,NaN,NaN,NaN,NaN,True False 2.0 nan F,NaN,NaN,NaN
001e2bba-90fa-4a62-bf39-ddb29a956d93,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0033fc70-2eb9-45f0-990e-d514bab75508,True False 3.0 0.0 W,True False 3.0 22.0 W,NaN,NaN,NaN,NaN,True False 3.0 12.0 W,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
00427b7c-fa09-430f-b505-4b04ad84f055,NaN,NaN,False False 1.0 69.0 P,NaN,NaN,NaN,NaN,NaN,NaN,False False 1.0 65.0 P,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
00458cb0-d9ce-4520-82d4-f14bc2a7a422,False False 1.0 45.0 P,True False 3.0 39.0 W,NaN,NaN,NaN,NaN,False False 1.0 47.0 P,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [13]:
RECORDS

{'00016413-5863-4441-af2c-9743f19bf6cd': ['201718 CONAD False False 1.0 51.0 P',
  '201718 CONTRACT True False 2.0 46.0 P',
  '201718 LSM True False 2.0 nan F'],
 '001e2bba-90fa-4a62-bf39-ddb29a956d93': ['201314 CONAD False False 1.0 54.0 P',
  '201314 CONTRACT False False 1.0 58.0 P',
  '201314 LSM False False 1.0 59.0 P',
  '201415 CRIMINAL False False 1.0 67.0 P',
  '201415 TORT False False 1.0 61.0 P',
  '201516 EQUITY False False 1.0 61.0 P',
  '201516 LAND False False 1.0 57.0 P',
  '201617 EU False True 1.0 61.0 P',
  '201617 LT1 False True 1.0 66.0 P',
  '201617 LT2 False True 1.0 62.0 P'],
 '0033fc70-2eb9-45f0-990e-d514bab75508': ['201112 CONAD True False 3.0 0.0 W',
  '201112 CONTRACT True False 3.0 22.0 W',
  '201112 LSM True False 3.0 12.0 W'],
 '00427b7c-fa09-430f-b505-4b04ad84f055': ['201112 CRIMINAL False False 1.0 69.0 P',
  '201112 TORT False False 1.0 65.0 P',
  '201213 EQUITY False False 1.0 67.0 P',
  '201314 EU False False 1.0 74.0 P',
  '201314 LT1 False False 1.0

In [14]:
# make dictionaries about student variables for inputting into a future 'STUDENTS' table (clean this up too)
year_entered = {}
graduated_year = {}
status_now = {}
STUDENTS = pd.DataFrame(columns=['Entire Record',
                                 'CONAD History','CONTRACT History', 'LSM History', 'TORT History', 
                                 'LAND History', 'CRIMINAL History','EQUITY History', 'EU History', 
                                 'LT1 History', 'LT2 History', 'CONAD Attempts', 'CONTRACT Attempts',
                                'LSM Attempts', 'TORT Attempts', 'LAND Attempts', 'CRIMINAL Attempts',
                                'EQUITY Attempts', 'EU Attempts', 'LT1 Attempts', 'LT2 Attempts'])

list_years = ['201112','201213','201314','201415','201516', '201617','201718']
for SPRcode, list_records in RECORDS.items(): # for all students...
    try:
        first_year = list_records[0].split()[0]
        last_year = list_records[-1].split()[0]
        # fill in entire record column
        STUDENTS.at[SPRcode, 'Entire Record'] = list_records
        # if left prematurely -> does not include any students that left after 2018
        if not any([(record.split()[0] == '201718') for record in list_records]):
            if not any([record.split()[1] == 'LT2' for record in list_records]):     
                left_prematurely_year_courses = []
                for record in list_records:
                    if record.split()[0] == last_year:
                        left_prematurely_year_courses.append((record.split()[1] + ' ' + record.split()[2]
                                                              + ' ' + record.split()[3] + ' ' + record.split()[4]
                                                              + ' ' + record.split()[5] + ' ' + record.split()[6]))
                # record last year and courses taken that year
#                 year_left_prematurely[SPRcode] = last_year
#                 courses_prior_to_leaving_prematurely[SPRcode] = left_prematurely_year_courses
                status_now[SPRcode] = 'Left prematurely'
        # if graduauted
        if any([((record.split()[1] == 'LT2') & (record.split()[-1] == 'P')) for record in list_records]):
            if any([((record.split()[1] == 'LT1') & (record.split()[-1] == 'P')) for record in list_records]):
                if any([((record.split()[1] == 'EU') & (record.split()[-1] == 'P')) for record in list_records]):
                    graduated_year[SPRcode] = last_year
                    status_now[SPRcode] = 'Graduated'
        # determine current status (progressing or not progressing) if not graduated or left
        if SPRcode not in status_now.keys():
            # determine if all courses passed for 201718
            if any([((record.split()[0] == '201718') & (record.split()[-1] == 'F')) for record in list_records]):
                status_now[SPRcode] = 'Not progressing'
            elif any([(record.split()[0] =='201718') for record in list_records]):
                status_now[SPRcode] = 'Progressing'
        # determine year entered if possible
        for year in list_years[1:]:
            if first_year == year:
                if any([f'{year} CONAD' in record for record in list_records]):
                    if any([f'{year} CONTRACT' in record for record in list_records]):
                        year_entered[SPRcode] = year
        if (first_year == '201112') & (SPRcode not in mod_files_2010[module].index):
            if any([f'{first_year} CONAD' in record for record in list_records]):
                if any([f'{first_year} CONTRACT' in record for record in list_records]):
                    year_entered[SPRcode] = first_year          
    except:
        print(SPRcode, list_records)
        
# find the PT/FT status of each student (clean this up or make function above)
student_pt_ft_statuses = pd.DataFrame()

for dfname, df in module_files.items():
    year, module, tmp = dfname.split('_')
    student_pt_ft_statuses = (student_pt_ft_statuses.join(df[['Programme']], how='outer'))
    student_pt_ft_statuses.rename(columns={'Programme': f'{year} {module}'}, inplace=True)

student_pt_ft_statuses = student_pt_ft_statuses.reindex(sorted(student_pt_ft_statuses.columns), axis=1)#sorted by year

records = {}
for index, row in student_pt_ft_statuses.iterrows():
    courses_on_record = row.dropna().index.tolist()
    programmes_listed = row.dropna().values.tolist()
    records[index] = list(zip(courses_on_record,programmes_listed))
    
changed_programme = {}
programmes = {}
for SPRcode, record in records.items():
    first_programme = record[0][1]
    for (course, programme) in record:
        if programme != first_programme:
            changed_programme[SPRcode] = record
    if SPRcode in changed_programme:
        programmes[SPRcode] = 'Changed'
    else:
        if 'part' in first_programme:
            programmes[SPRcode] = 'PT'
        elif 'full' in first_programme:
            programmes[SPRcode] = 'FT'  
        elif '6' in first_programme:
            programmes[SPRcode] = '6YR'

In [16]:
# Make STUDENTS dataframe
STUDENTS['Programme'] = pd.Series(programmes)
STUDENTS['Year started'] = pd.Series(year_entered)
STUDENTS['Current status'] = pd.Series(status_now)
STUDENTS['Graduated year'] = pd.Series(graduated_year)
# STUDENTS['Year left prematurely'] = pd.Series(year_left_prematurely)
# STUDENTS['Courses prior to leaving prematurely'] = pd.Series(courses_prior_to_leaving_prematurely)
cols = ['Programme','Year started', 'Current status', 'Graduated year', 'Entire Record', 
        'LSM History', 'CONTRACT History', 'CONAD History','TORT History',  'CRIMINAL History', 'LAND History',
       'EQUITY History', 'EU History', 'LT1 History', 'LT2 History', 'LSM Attempts', 'CONTRACT Attempts',
                                 'CONAD Attempts', 'TORT Attempts', 'CRIMINAL Attempts', 'LAND Attempts',
                                'EQUITY Attempts', 'EU Attempts', 'LT1 Attempts', 'LT2 Attempts']
STUDENTS = STUDENTS[cols]

In [17]:
# check that all 
any(STUDENTS['Entire Record'].isnull())

False

In [18]:
# determine the last time that the course shows up, and the attempt number of that time 
# also could count the number of attempts myself
for name in ['CONAD History','CONTRACT History', 'LSM History', 'TORT History', 
            'LAND History', 'CRIMINAL History','EQUITY History', 'EU History', 
            'LT1 History', 'LT2 History']:
    STUDENTS[name] = np.empty((len(STUDENTS),0)).tolist()

def record_history(row):
    for record in row['Entire Record']:
        print(record)
        year, course, reassess_flag, dr_flag, attemptnum, mark, result = record.split()
        row[f'{course} History'].append(record)

STUDENTS[STUDENTS['Year started'].notnull()].apply(record_history, axis=1);

201718 CONAD False False 1.0 51.0 P
201718 CONTRACT True False 2.0 46.0 P
201718 LSM True False 2.0 nan F
201314 CONAD False False 1.0 54.0 P
201314 CONTRACT False False 1.0 58.0 P
201314 LSM False False 1.0 59.0 P
201415 CRIMINAL False False 1.0 67.0 P
201415 TORT False False 1.0 61.0 P
201516 EQUITY False False 1.0 61.0 P
201516 LAND False False 1.0 57.0 P
201617 EU False True 1.0 61.0 P
201617 LT1 False True 1.0 66.0 P
201617 LT2 False True 1.0 62.0 P
201112 CONAD True False 3.0 0.0 W
201112 CONTRACT True False 3.0 22.0 W
201112 LSM True False 3.0 12.0 W
201112 CONAD False False 1.0 45.0 P
201112 CONTRACT True False 3.0 39.0 W
201112 LSM False False 1.0 47.0 P
201314 CONAD False False 1.0 41.0 P
201314 CONTRACT False False 1.0 48.0 P
201314 LSM False False 1.0 49.0 P
201314 TORT True False 3.0 48.0 P
201415 CRIMINAL False False 1.0 45.0 P
201415 EQUITY False False 1.0 40.0 P
201415 LAND False False 1.0 52.0 P
201516 EU True False 3.0 40.0 P
201516 LT1 False False 1.0 52.0 P
201516 L

201617 LAND False False 1.0 47.0 P
201718 EU False False 1.0 61.0 P
201718 LT1 False False 1.0 58.0 P
201718 LT2 False False 1.0 48.0 P
201314 CONAD False False 1.0 0.0 W
201314 CONTRACT False False 1.0 0.0 W
201314 LSM False False 1.0 0.0 W
201314 TORT False False 1.0 0.0 W
201415 CONAD False False 1.0 52.0 P
201415 CONTRACT False False 1.0 50.0 P
201415 LSM False False 1.0 nan P
201415 TORT False False 1.0 42.0 P
201516 CRIMINAL False False 1.0 50.0 P
201516 EQUITY True False 2.0 40.0 P
201516 LAND False False 1.0 50.0 P
201617 EU False True 1.0 65.0 P
201617 LT1 False False 1.0 61.0 P
201617 LT2 False True 1.0 50.0 P
201415 CONAD True False 3.0 36.0 F
201415 CONTRACT False False 1.0 50.0 P
201415 LSM True False 2.0 nan F
201415 TORT True False 3.0 28.0 F
201112 CONAD False False 1.0 66.0 P
201112 CONTRACT False False 1.0 63.0 P
201112 LSM False False 1.0 62.0 P
201213 CRIMINAL False False 1.0 0.0 W
201213 TORT False False 1.0 0.0 W
201314 CRIMINAL False False 1.0 42.0 P
201314 TORT 

201415 CONTRACT False False 1.0 45.0 P
201415 LSM False False 1.0 nan P
201415 TORT False True 1.0 55.0 P
201516 CRIMINAL False True 1.0 0.0 F
201516 EQUITY True False 2.0 0.0 F
201516 LAND False False 1.0 43.0 P
201617 CRIMINAL False False 2.0 66.0 P
201617 EQUITY False False 3.0 0.0 F
201617 CONAD False False 1.0 62.0 P
201617 CONTRACT False False 1.0 61.0 P
201617 LSM False False 1.0 nan P
201718 CRIMINAL False False 1.0 59.0 P
201718 TORT False False 1.0 62.0 P
201718 CONAD True False 2.0 37.0 F
201718 CONTRACT False True 1.0 21.0 W
201718 LSM False False 1.0 nan P
201718 TORT True False 2.0 0.0 F
201213 CONAD False False 1.0 0.0 W
201213 CONTRACT False False 1.0 0.0 W
201213 LSM False False 1.0 0.0 W
201213 CONAD True False 3.0 34.0 F
201213 CONTRACT False False 1.0 56.0 P
201213 LSM False False 1.0 50.0 P
201516 CONAD False False 1.0 0.0 F
201516 CONTRACT False False 1.0 28.0 F
201516 LSM False False 1.0 nan F
201617 CONAD False False 2.0 41.0 P
201617 CONTRACT False False 2.0 46

201314 LT2 False False 1.0 72.0 P
201213 CONAD False False 1.0 0.0 W
201213 CONTRACT False False 1.0 0.0 W
201213 LSM False False 1.0 0.0 W
201112 CONAD False False 1.0 64.0 P
201112 CONTRACT False False 1.0 50.0 P
201112 LSM False False 1.0 69.0 P
201213 CRIMINAL False False 1.0 68.0 P
201213 LAND False False 1.0 44.0 P
201213 TORT False False 1.0 46.0 P
201314 EQUITY False False 1.0 48.0 P
201314 LAND False False 1.0 44.0 P
201415 EU False False 1.0 68.0 P
201415 LT1 False False 1.0 61.0 P
201415 LT2 False False 1.0 68.0 P
201213 CONAD False False 1.0 0.0 W
201213 CONTRACT False False 1.0 0.0 W
201213 LSM False False 1.0 0.0 W
201314 CONAD False False 2.0 0.0 W
201314 CONTRACT False False 2.0 0.0 W
201314 LSM False False 2.0 0.0 W
201213 CONAD False False 1.0 0.0 F
201213 CONTRACT False False 1.0 0.0 F
201213 LSM False False 1.0 0.0 F
201314 CONAD False False 2.0 0.0 W
201314 CONTRACT False False 2.0 0.0 W
201314 LSM False False 2.0 0.0 W
201516 CONAD True False 3.0 0.0 W
201516 CONT

201314 EQUITY False False 1.0 53.0 P
201314 LAND False False 1.0 66.0 P
201415 EU False False 1.0 63.0 P
201415 LT1 False False 1.0 58.0 P
201415 LT2 False False 1.0 69.0 P
201617 CONAD False False 1.0 41.0 P
201617 CONTRACT True False 2.0 0.0 F
201617 LSM True False 3.0 nan F
201617 TORT False False 1.0 41.0 P
201415 CONAD False True 3.0 1.0 W
201415 CONTRACT False True 3.0 0.0 W
201415 LSM True False 2.0 nan F
201415 TORT False True 3.0 0.0 W
201516 LSM False True 3.0 nan F
201314 CONAD False False 1.0 51.0 P
201314 CONTRACT True False 3.0 38.0 CF
201314 LSM False False 1.0 49.0 P
201314 TORT True False 2.0 44.0 P
201415 CRIMINAL False False 1.0 41.0 P
201415 EQUITY False False 1.0 46.0 P
201415 LAND True False 3.0 56.0 P
201617 EU False False 1.0 58.0 P
201617 LT1 False False 1.0 62.0 P
201617 LT2 False False 1.0 63.0 P
201415 CONAD True False 2.0 29.0 F
201415 CONTRACT True False 3.0 58.0 P
201415 LSM False False 1.0 nan W
201516 LSM False False 1.0 nan P
201718 CONAD False False 3

201213 CRIMINAL False False 1.0 70.0 P
201213 TORT False True 1.0 61.0 P
201314 EQUITY False True 1.0 41.0 P
201314 EU False True 1.0 63.0 P
201314 LT1 False False 1.0 63.0 P
201314 LT2 False False 1.0 71.0 P
201718 CONAD False False 1.0 64.0 P
201718 CONTRACT False False 1.0 64.0 P
201718 LSM False False 1.0 nan P
201718 TORT False False 1.0 57.0 P
201617 CONAD False True 1.0 71.0 P
201617 CONTRACT False False 1.0 62.0 P
201617 LSM False False 1.0 nan P
201617 TORT False True 1.0 68.0 P
201718 CRIMINAL False False 1.0 65.0 P
201718 EQUITY False False 1.0 63.0 P
201718 LAND False False 1.0 60.0 P
201314 CONAD True False 3.0 35.0 F
201314 CONTRACT True False 3.0 21.0 F
201314 LSM True False 3.0 9.0 EF
201314 TORT True False 3.0 27.0 F
201415 LSM True False 2.0 nan F
201516 LSM False False 3.0 nan P
201718 CONAD False False 1.0 58.0 P
201718 CONTRACT False False 1.0 42.0 P
201718 LSM False False 1.0 nan P
201718 TORT True False 2.0 36.0 CF
201415 CONAD True False 3.0 0.0 F
201415 CONTRAC

201213 TORT False False 1.0 0.0 F
201314 CRIMINAL True False 3.0 39.0 W
201314 TORT True False 3.0 21.0 W
201415 TORT False False 3.0 0.0 W
201516 CONAD False False 1.0 64.0 P
201516 CONTRACT False False 1.0 63.0 P
201516 LSM False False 1.0 nan P
201516 TORT False False 1.0 56.0 P
201617 CRIMINAL False False 1.0 61.0 P
201617 EQUITY False False 1.0 58.0 P
201617 LAND False False 1.0 58.0 P
201718 EU False False 1.0 61.0 P
201718 LT1 False False 1.0 68.0 P
201718 LT2 False False 1.0 72.0 P
201213 CONAD False False 1.0 65.0 P
201213 CONTRACT False False 1.0 68.0 P
201213 LSM False False 1.0 76.0 P
201314 CRIMINAL False False 1.0 63.0 P
201314 TORT False False 1.0 52.0 P
201415 EQUITY False False 1.0 48.0 P
201415 LAND False False 1.0 63.0 P
201516 EU False True 1.0 73.0 P
201516 LT1 False True 1.0 68.0 P
201516 LT2 False True 1.0 60.0 P
201718 CONAD True False 2.0 40.0 P
201718 CONTRACT False False 1.0 41.0 P
201718 LSM True False 2.0 nan P
201112 CONAD False False 1.0 47.0 P
201112 CON

In [19]:
# this is what I need to fix -> how to make parse dr and fr attempts properly 
# and also catch where the administration didn't 

for name in ['LSM Attempts','CONTRACT Attempts','CONAD Attempts', 'TORT Attempts', 
        'CRIMINAL Attempts','LAND Attempts', 'EQUITY Attempts', 'EU Attempts', 
        'LT1 Attempts', 'LT2 Attempts']:
    STUDENTS[name] = np.empty((len(STUDENTS),0)).tolist()

for course in ['CONAD','CONTRACT','LSM','TORT','EQUITY','LAND','CRIMINAL','EU','LT1','LT2']:
    for index, row in STUDENTS.iterrows():
        for record in row[f'{course} History']:
            year, course, reassess_flag, dr_flag, attemptnum, mark, result = record.split()
            next_year = year.split('_')[0][:2] + \
                        str(int(year.split('_')[0][2:4]) + 1) + \
                        str(int(year.split('_')[0][4:]) + 1)
            previous_year = year.split('_')[0][:2] + \
                        str(int(year.split('_')[0][2:4]) - 1) + \
                        str(int(year.split('_')[0][4:]) - 1)
            if index not in mod_files_2010[module].index: 
                # only calculating the attempt nums 
                # for students that entered 2011 or later
                attemptnum = int(float(attemptnum)) #2
                if any(row[f'{course} Attempts']): # any attempts already for this course and student id
                    if type(row[f'{course} Attempts'][-1][-1]) == list:  # if there were multiple previous attempts
                        most_recent_previous_attempt = float(row[f'{course} Attempts'][-1][-1][-1])
                    else: # if there was a single previous attempt   
                        most_recent_previous_attempt = float(row[f'{course} Attempts'][-1][-1])
                    attempt_diff = int(float(attemptnum) - most_recent_previous_attempt)
                        #subtract the most recent previous attempt
                    if attempt_diff == 0:
                        previous_year_name = previous_year + '_' + module + '_' + 'assessment'
                        try:
                            previous_year_result = module_files[previous_year_name].at[index,'Result']
                        except:
                            previous_year_result = np.NaN
                        if previous_year_result == 'D':
                            if attemptnum == 1:
                                attemptnum = int(most_recent_previous_attempt)
                            elif (reassess_flag == 'True'): 
                                # could add 2 or 3 attempts here, depending whether there is a third assessment the following year
                                attemptnum = int(most_recent_previous_attempt) + attemptnum
                        else: # catch where attemptnum doesn't increment
                            attemptnum = int(most_recent_previous_attempt) + attemptnum
                else: # if no previous attempts recorded
                    attempt_diff = int(float(attemptnum)) 
                if dr_flag == 'True': # student supposed to have had one official attempt that year, but sometimes this doesn't happen 
                    if (attempt_diff == 3): # admin counted DR as an attempt
                        row[f'{course} Attempts'].append((year, ['' ,''], [attemptnum-2, attemptnum-1]))
                        row[f'{course} Attempts'].append((next_year, result, attemptnum))
                    elif (attempt_diff == 2):
                        row[f'{course} Attempts'].append((year, ['', result], [attemptnum-1, attemptnum]))
                    else:
                        #the DR was not counted by admin but had an effect that should be recorded
                        row[f'{course} Attempts'].append((year, ['', result], [attemptnum, attemptnum]))
                elif (reassess_flag == 'True'): # if attempt that year included an FR
                    if (attempt_diff == 3): # if the difference between the previous attempt and this one is 3
                        # there is an attempt and a reassessment that year
                        # and another reassessment the next year that isn't recorded there      
                        row[f'{course} Attempts'].append((year, ['', ''], [attemptnum-2, attemptnum-1]))
                        row[f'{course} Attempts'].append((next_year, result, attemptnum))
                    elif (attempt_diff == 2):
                        row[f'{course} Attempts'].append((year, ['', result], [attemptnum-1, attemptnum]))
                    elif (attempt_diff == 1):
                        print('attempt_diff error', index, record, '*', row[f'{course} History'])
                        row[f'{course} Attempts'].append((year, result, attemptnum+1))
                else:
                    row[f'{course} Attempts'].append((year, result, attemptnum))
            else:
                row[f'{course} Attempts'] = []

attempt_diff error 59962c35-ca36-4e95-9459-6e5a252b0902 201718 CONAD True False 2.0 40.0 P * ['201112 CONAD False False 1.0 53.0 P', '201718 CONAD True False 2.0 40.0 P']
attempt_diff error 6e74f9e4-ff33-4547-8b17-78ff7b7fac45 201314 CONAD True False 2.0 0.0 F * ['201213 CONAD False False 1.0 0.0 W', '201314 CONAD True False 2.0 0.0 F', '201415 CONAD False False 3.0 0.0 F']
attempt_diff error 7464269a-6218-4124-88b6-8b9f89116993 201516 CONAD True False 3.0 0.0 W * ['201213 CONAD False False 1.0 0.0 F', '201314 CONAD False False 2.0 0.0 W', '201516 CONAD True False 3.0 0.0 W']
attempt_diff error cd6997db-b179-4ae3-ae1b-e83edf502318 201415 CONAD True False 3.0 37.0 CF * ['201213 CONAD True False 2.0 3.0 W', '201415 CONAD True False 3.0 37.0 CF']
attempt_diff error d453e727-2aa4-4562-8f61-47243a927d2e 201617 CONAD True False 3.0 0.0 F * ['201516 CONAD True False 2.0 0.0 W', '201617 CONAD True False 3.0 0.0 F']
attempt_diff error e47a274a-a67a-455c-a3b7-79c35b2ad66b 201314 CONAD True False

attempt_diff error 2ff4112b-80aa-49fb-9fb5-6d0a2fe0dc29 201718 TORT True False 2.0 40.0 P * ['201617 TORT False False 1.0 0.0 W', '201718 TORT True False 2.0 40.0 P']
attempt_diff error bcb36a16-bbf3-47a3-9838-276ee06bf3df 201718 TORT True False 3.0 0.0 F * ['201314 TORT False True 1.0 5.0 W', '201617 TORT False False 1.0 0.0 F', '201718 TORT True False 3.0 0.0 F']
attempt_diff error c8a380f6-ad1f-408a-a77a-aa87942c1b08 201617 TORT True False 2.0 0.0 F * ['201415 TORT False False 1.0 0.0 W', '201617 TORT True False 2.0 0.0 F']
attempt_diff error 19229382-32b9-4ce2-95aa-32c11e3fb121 201718 EQUITY True False 3.0 25.0 F * ['201516 EQUITY False False 1.0 0.0 F', '201617 EQUITY False True 2.0 0.0 W', '201718 EQUITY True False 3.0 25.0 F']
attempt_diff error 2034d6c4-4182-42ec-accb-cfc6d5ef943f 201415 EQUITY True False 1.0 41.0 P * ['201415 EQUITY True False 1.0 41.0 P']
attempt_diff error 6d1c2cbd-aed5-482a-af26-76009cf22927 201718 EQUITY True False 3.0 29.0 F * ['201516 EQUITY False True 1

In [ ]:
year, result, attemptnum

In [ ]:
# STUDENTS.drop(labels=['Year 1 Courses', 'Year 2 Courses', 
#         'Year 3 Courses', 'Year 4 Courses', 'Year 5 Courses', 'Year 6 Courses'], axis=1, inplace=True)

In [20]:
STUDENTS.to_excel("/Users/Kate/Desktop/Vicky project/data/STUDENTS.xls")

In [21]:
STUDENTS['Programme'].value_counts()

PT         938
FT         869
Changed     21
6YR         20
Name: Programme, dtype: int64

In [22]:
# Calculate metrics and create visualizations

# what proportion of students graduated within 6 years (PT) or 5 years (FT) for students starting in 2011, 2012, 2013?
# write function or loop to analyze each programme by year entered (2 by 3)

years_entered = pd.Index(['2011', '2012'])
programmes = pd.Index(['FT', 'PT'])

graduation_rate_df = pd.DataFrame(index=programmes, columns=years_entered) 

for programme in programmes.tolist():
    for year in years_entered.tolist():
        if programme == 'PT':
            cutoff = int(year) + 6
        elif programme == 'FT':
            cutoff = int(year) + 5
        selection_total_students = STUDENTS['Year started'].str.contains(year).fillna(False) \
                                    & (STUDENTS['Programme'] == programme)
        selection_graduated_students = (selection_total_students 
                                        & (STUDENTS.loc[STUDENTS['Graduated year'].notnull(), 'Graduated year']
                                           .apply(lambda x: x[:2] + x[-2:])
                                           .astype('int') 
                                           <= cutoff))
        graduation_rate_df.at[programme, year] = (len(STUDENTS[selection_graduated_students]) 
                                                  / len(STUDENTS[selection_total_students]))
graduation_rate_df

,2011,2012
FT,0.571429,0.447619
PT,0.472637,0.384615


In [23]:
# what proportion of students graduated within 6 years (PT) or 5 years (FT) for students starting in 2011, 2012, 2013?
# write function or loop to analyze each programme by year entered (2 by 3)

###############################################################################
'''
this isn't quite right, because the 3rd attempts are actually the following year
'''
###############################################################################

print('Leaving early rate: ')
years_entered = pd.Index(['2011', '2012'])
programmes = pd.Index(['FT', 'PT'])

leaving_rate_df = pd.DataFrame(index=programmes, columns=years_entered)

for programme in programmes.tolist():
    for year in years_entered.tolist():
        if programme == 'PT':
            cutoff = int(year) + 6
        elif programme == 'FT':
            cutoff = int(year) + 5
        selection_total_students = STUDENTS['Year started'].str.contains(year).fillna(False) \
                                    & (STUDENTS['Programme'] == programme)
        selection_left_students = (selection_total_students 
                                        & (STUDENTS.loc[STUDENTS['Year left prematurely'].notnull(), 'Year left prematurely']
                                           .apply(lambda x: x[:2] + x[-2:])
                                           .astype('int') 
                                           <= cutoff))
        leaving_rate_df.at[programme, year] = (len(STUDENTS[selection_left_students]) 
                                                  / len(STUDENTS[selection_total_students]))
leaving_rate_df

Leaving early rate: 


KeyError: 'Year left prematurely'

In [24]:
# calculate time to leaving
###############################################################################
'''
this isn't quite right, because the 3rd attempts are actually the following year
'''
###############################################################################
print('Time spent before leaving early: ')
for programme in programmes.tolist():
    df = (STUDENTS.loc[(STUDENTS['Year left prematurely'].notnull() 
                        & (STUDENTS['Programme'] == programme)), 'Year left prematurely']
                  .apply(lambda x: x[:2] + x[-2:])
                  .astype('int') 
        - STUDENTS.loc[(STUDENTS['Year started'].notnull() 
                        & (STUDENTS['Programme'] == programme)), 'Year started']
          .apply(lambda x: x[:4])
          .astype('int'))
    time = df.mean()
    print(programme, time)


Time spent before leaving early: 


KeyError: 'Year left prematurely'

In [25]:
# calculate time to completion
print('Time to completion: ')
for programme in programmes.tolist():
    df = (STUDENTS.loc[(STUDENTS['Graduated year'].notnull() & (STUDENTS['Programme'] == programme)), 'Graduated year']
                  .apply(lambda x: x[:2] + x[-2:])
                  .astype('int') 
        - STUDENTS.loc[(STUDENTS['Year started'].notnull() & (STUDENTS['Programme'] == programme)), 'Year started']
          .apply(lambda x: x[:4])
          .astype('int'))
    time = df.mean()
    print(programme, time)

Time to completion: 
FT 3.249158249158249
PT 4.206896551724138


In [26]:
for dfname, df in module_files.items():
    module_files[dfname]['Real Attempt Number(s)'] = np.empty((len(df),0)).tolist()

for course in ['CONAD','CONTRACT','LSM','TORT','EQUITY','LAND','CRIMINAL','EU','LT1','LT2']:
    for index, row in STUDENTS.iterrows():
        if any(row[f'{course} Attempts']):
            for record in row[f'{course} Attempts']:
                year = record[0]
                if year == '201819':
                    print(index + ' year 201819')
                    pass
                else:
                    attempts_that_year = record[-1]
                    module_files[f'{year}_{course}_assessment'].at[index,'Real Attempt Number(s)'] = attempts_that_year
                    # real attempt number is either empty list, integer, or list
                    # let's replace the empty lists with np.NaN

for dfname, df in module_files.items():
    for index, row in df.iterrows(): 
        if type(row['Real Attempt Number(s)']) == list:
            if not any(row['Real Attempt Number(s)']):
                module_files[dfname].at[index, 'Real Attempt Number(s)'] = np.NaN
                
dfname

78cf4aae-b989-44e9-a40a-e64fa78b2420 year 201819
ade3cd0a-84ae-44b2-9909-6c251e65c4d0 year 201819


'201516_LSM_assessment'

In [ ]:
module_files[f'{year}_{course}_assessment'].to_excel("/Users/Kate/Desktop/Vicky project/data/example_moduledf.xls")

In [ ]:
year = '201314'
course = 'CONTRACT'
module_files[f'{year}_{course}_assessment'].sort_index()

In [ ]:
module_files[f'{year}_{course}_assessment'].columns

In [ ]:
assignment_numbers = {}
for dfname, df in module_files.items():
    assignment_numbers[dfname] = df.columns[-4].split()[1]
max_num_assignments = int(max(assignment_numbers.values())) #2
assignment_numbers;

In [27]:
ATTEMPTS = {}
tuples = []
for index in STUDENTS.index:
    for course in  ['CONAD', 'CONTRACT', 'LSM', 'TORT', 'LAND', 'CRIMINAL', 'EQUITY', 'EU', 'LT1', 'LT2']:
        tuples.append((index,course))

index = pd.MultiIndex.from_tuples(tuples, names=['SPRcode', 'Module'])
        
for attempt in ['first', 'second', 'third', 'fourth', 'fifth']:
    ATTEMPTS[f'{attempt}'] = pd.DataFrame(index=index, columns = ['Student Programme','Student Final Status',
                                                    'Year of Attempt', 'Attempt Type', 'Module Mark', 
                                                     'Module Grade', 'Module Result'])

first_counter = 0
for dfname, df in module_files.items():
    year, module, tmp = dfname.split('_') 
    for index, row in df.iterrows():
        previous_year_name = year.split('_')[0][:2] + \
                        str(int(year.split('_')[0][2:4]) - 1) + \
                        str(int(year.split('_')[0][4:]) - 1) + '_' + module + '_' + 'assessment'
        next_year_name = year.split('_')[0][:2] + \
                        str(int(year.split('_')[0][2:4]) + 1) + \
                        str(int(year.split('_')[0][4:]) + 1) + '_' + module + '_' + 'assessment'
        try:
            previous_year_result = module_files[previous_year_name].at[index,'Result']
        except:
            previous_year_result = np.NaN 
        try:
            if index in module_files[next_year_name].index:
                if type(module_files[next_year_name].at[index,'Result']) != str:
                    # this year is a reassessment that was failed
                    second_reassessment_flag = True #worked for '0033fc70-2eb9-45f0-990e-d514bab75508'&'201112_CONAD_assessment'
                else:
                    second_reassessment_flag = False
            else:
                second_reassessment_flag = False             
        except:
            second_reassessment_flag = False # False -> took except route

        ####type(row['Result']) # str

        ####################################################################
        if (type(row['Result']) == str): 
            # if this isn't the second reassessment (which is recorded independently)
            primary_row = pd.Series()
            primary_row['Student Programme'] = STUDENTS.at[index,'Programme'] # PT
            primary_row['Student Final Status'] = STUDENTS.at[index, 'Current status'] # not progressing
            primary_row['Year of Attempt'] = year # 201718
            ####################################################################
             # if a reassessment
            ####################################################################
            if (row['Reassess Flag'] == True) | (row['DR Flag'] == True): # if the year includes a reassessment 
                if second_reassessment_flag == False:
                    reassessment_row = primary_row.copy() # otherwise will overwrite primary_row!!!
                    reassessment_row['Module Grade'] = row['Grade']
                    reassessment_row['Module Result'] = row['Result'] 
                    if any(df.columns == 'Mark'):
                        reassessment_row['Module Mark'] = row['Mark']
                    else:
                        reassessment_row['Module Mark'] = np.NaN
                    if row['DR Flag'] == True:
                        reassessment_row['Attempt Type'] = 'DR Reassessment'
                    else:
                        reassessment_row['Attempt Type'] = 'FR Reassessment'
                    try:
                        if type(row['Real Attempt Number(s)']) == list:
                            reassessment_attempt_num = row['Real Attempt Number(s)'][-1]
                        else:
                            first_attempt_that_year = row['Real Attempt Number(s)']
                        if reassessment_attempt_num == 1: # DR reassessment
                            ATTEMPTS['second'].loc[(index, module)] = reassessment_row
                        if reassessment_attempt_num == 2:
                            ATTEMPTS['second'].loc[(index, module)] = reassessment_row 
                        if reassessment_attempt_num == 3:
                            ATTEMPTS['third'].loc[(index,module)] = reassessment_row
                        if reassessment_attempt_num == 4:
                            ATTEMPTS['fourth'].loc[(index,module)] = reassessment_row
                        if reassessment_attempt_num == 5:
                            ATTEMPTS['fifth'].loc[(index,module)] = reassessment_row
                    except:
                        # if there is no attempt number (ie started before 2011), then we don't care to write it anyway
                        if type(STUDENTS.at[index,'Year started']) == str:
                            print(STUDENTS.at[index,'Year started'],'no attempt number', index, module, year)
                        pass
                elif second_reassessment_flag == True:
                    # will need to make a row each to put in first reassessment and second reassessment
                    first_reassessment_row = primary_row.copy() # do not overwrite primary_row
                    second_reassessment_row = primary_row.copy() # do not overwrite primary_row or first_assessment_row
                    first_reassessment_row['Module Grade'] = np.NaN
                    second_reassessment_row['Module Grade'] = row['Grade'] 
                    first_reassessment_row['Module Result'] = 'F calc?'
                    second_reassessment_row['Module Result'] = row['Result'] 
                    first_reassessment_row['Module Mark'] = np.NaN
                    if any(df.columns == 'Mark'):
                        second_reassessment_row['Module Mark'] = row['Mark']
                    else:
                        second_reassessment_row['Module Mark'] = np.NaN
                    if (row['DR Flag'] == True):
                        first_reassessment_row['Attempt Type'] = 'DR Reassessment'
                    else:
                        first_reassessment_row['Attempt Type'] = 'FR Reassessment'
                    second_reassessment_row['Attempt Type'] = 'Reassessment Following Year'
                    if type(row['Real Attempt Number(s)']) == list:
                        first_reassessment_attempt_num = row['Real Attempt Number(s)'][-1]
                    else:
                        first_reassessment_attempt_num = row['Real Attempt Number(s)']
                    if first_reassessment_attempt_num == 1:
                        print('error with attemptnum = 1 after reassessment',index, module, year)
                    if first_reassessment_attempt_num == 2:
                        ATTEMPTS['second'].loc[(index, module)] = first_reassessment_row
                        ATTEMPTS['third'].loc[(index, module)] = second_reassessment_row
                    if first_reassessment_attempt_num == 3:
                        ATTEMPTS['third'].loc[(index,module)] = first_reassessment_row
                        ATTEMPTS['fourth'].loc[(index, module)] = second_reassessment_row
                    if first_reassessment_attempt_num == 4:
                        ATTEMPTS['fourth'].loc[(index,module)] = first_reassessment_row
                        ATTEMPTS['fifth'].loc[(index, module)] = second_reassessment_row
                    if first_reassessment_attempt_num == 5:
                        ATTEMPTS['fifth'].loc[(index,module)] = first_reassessment_row
                    ####################################################################
                    # put primary rows in correct attempt dfs
                    ####################################################################
                if not ((pd.to_numeric(year,errors='coerce') > 201314) & (module =='LSM')): #needs testing
                    try:
                        if (int(df.columns[-4].split()[1]) == 2): # num assignments
                            primary_row['Module Mark'] = float((pd.to_numeric(row['Assessment 1 Weight']) 
                                                                * pd.to_numeric(row['Assessment 1 Mark']))
                                                               + (pd.to_numeric(row['Assessment 2 Weight']) 
                                                                * pd.to_numeric(row['Assessment 2 Mark'])))
                        else:
                            primary_row['Module Mark'] = row['Assessment 1 Mark']
                        if int(float(pd.to_numeric(primary_row['Module Mark']))) >= 40.0:
                            primary_row['Module Grade'] = 'P calc'
                            primary_row['Module Result'] = 'P calc'
                        else:
                            primary_row['Module Grade'] = 'F calc'
                            primary_row['Module Result'] = 'F calc'    
                    except:
                        print('error in calculating mark',index, module, year, row['Assessment 1 Mark'])
                        # this is a reassessment that is taking the full year
                        # perhaps is being recorded below
                else: # course is LSM and both assignments must be passed to pass
                    primary_row['Module Mark'] = np.NaN
                    if ((df.at[index,'Assessment 1 Grade'] in ['P','LP']) & (
                        df.at[index,'Assessment 2 Grade'] in ['P','LP'])):
                        primary_row['Module Grade'] = 'P calc'
                        primary_row['Module Result'] = 'P calc'
                    else:
                        primary_row['Module Grade'] = 'F calc'
                        primary_row['Module Result'] = 'F calc'
            ####################################################################
             # if no reassessment
            ####################################################################
            else: # no reassessment that year
                primary_row['Module Grade'] = row['Grade']
                primary_row['Module Result'] = row['Result'] 
                if any(df.columns == 'Mark'):
                    primary_row['Module Mark'] = row['Mark']
                else:
                    primary_row['Module Mark'] = np.NaN

            ####################################################################
             # store the first primary row
            ####################################################################
            if type(row['Real Attempt Number(s)']) == list:
                first_attempt_that_year = row['Real Attempt Number(s)'][0]
            else:
                first_attempt_that_year = row['Real Attempt Number(s)']
            if (first_attempt_that_year == 1): # initial assessment
                first_counter +=1
                if previous_year_result != 'D':
                    primary_row['Attempt Type'] = 'Initial Assessment'
                    ATTEMPTS['first'].loc[(index, module)] = primary_row
                else:
                    primary_row['Attempt Type'] = 'DR Retake'
                    ATTEMPTS['second'].loc[(index, module)] = primary_row
            else:# make retake
                retake_attempt_num = first_attempt_that_year
                primary_row['Attempt Type'] = 'Retake'
                if retake_attempt_num == 2:
                    if previous_year_result != 'D':
                        ATTEMPTS['second'].loc[(index, module)] = primary_row
                    else:
                        primary_row['Attempt Type'] = 'DR Retake'
                        ATTEMPTS['third'].loc[(index, module)] = primary_row
                elif retake_attempt_num == 3:
                    if previous_year_result != 'D':
                        ATTEMPTS['third'].loc[(index,module)] = primary_row
                    else:
                        primary_row['Attempt Type'] = 'DR Retake'
                        ATTEMPTS['fourth'].loc[(index, module)] = primary_row     
                elif retake_attempt_num == 4:
                    if previous_year_result != 'D':
                        ATTEMPTS['fourth'].loc[(index,module)] = primary_row
                    else:
                        primary_row['Attempt Type'] = 'DR Retake'
                        ATTEMPTS['fifth'].loc[(index, module)] = primary_row
                elif retake_attempt_num == 5:
                    ATTEMPTS['fifth'].loc[(index,module)] = primary_row 

In [28]:
first_counter

9069

In [29]:
for attempt in ['first', 'second', 'third', 'fourth', 'fifth']:
    ATTEMPTS[f'{attempt}'].dropna(how='all', inplace = True) 
    ATTEMPTS[f'{attempt}']['Module Mark'] = pd.to_numeric(ATTEMPTS[f'{attempt}']['Module Mark'])

In [30]:
ATTEMPTS['first']['Attempt Type'].value_counts(dropna=False)

Initial Assessment    9027
Name: Attempt Type, dtype: int64

In [31]:
first_attempts = ATTEMPTS['first'].copy()
first_attempts.reset_index(inplace=True)
first_attempts

,SPRcode,Module,Student Programme,Student Final Status,Year of Attempt,Attempt Type,Module Mark,Module Grade,Module Result
0,00016413-5863-4441-af2c-9743f19bf6cd,CONAD,PT,Not progressing,201718,Initial Assessment,51.0,P,P
1,00016413-5863-4441-af2c-9743f19bf6cd,CONTRACT,PT,Not progressing,201718,Initial Assessment,38.0,F calc,F calc
2,00016413-5863-4441-af2c-9743f19bf6cd,LSM,PT,Not progressing,201718,Initial Assessment,NaN,F calc,F calc
3,001e2bba-90fa-4a62-bf39-ddb29a956d93,CONAD,PT,Graduated,201314,Initial Assessment,54.0,P,P
4,001e2bba-90fa-4a62-bf39-ddb29a956d93,CONTRACT,PT,Graduated,201314,Initial Assessment,58.0,P,P
5,001e2bba-90fa-4a62-bf39-ddb29a956d93,LSM,PT,Graduated,201314,Initial Assessment,59.0,P,P
6,001e2bba-90fa-4a62-bf39-ddb29a956d93,TORT,PT,Graduated,201415,Initial Assessment,61.0,P,P
7,001e2bba-90fa-4a62-bf39-ddb29a956d93,LAND,PT,Graduated,201516,Initial Assessment,57.0,P,P
8,001e2bba-90fa-4a62-bf39-ddb29a956d93,CRIMINAL,PT,Graduated,201415,Initial Assessment,67.0,P,P
9,001e2bba-90fa-4a62-bf39-ddb29a956d93,EQUITY,PT,Graduated,201516,Initial Assessment,61.0,P,P


In [32]:
ATTEMPTS['second']['Attempt Type'].value_counts(dropna=False)

FR Reassessment    1941
DR Reassessment    1095
Retake              422
Name: Attempt Type, dtype: int64

In [33]:
second_attempts = ATTEMPTS['second'].copy()
second_attempts

Student Programme  \
SPRcode                              Module                       
00016413-5863-4441-af2c-9743f19bf6cd CONTRACT                PT   
                                     LSM                     PT   
001e2bba-90fa-4a62-bf39-ddb29a956d93 EU                      PT   
                                     LT1                     PT   
                                     LT2                     PT   
0033fc70-2eb9-45f0-990e-d514bab75508 CONAD                   PT   
                                     CONTRACT                PT   
                                     LSM                     PT   
00458cb0-d9ce-4520-82d4-f14bc2a7a422 CONTRACT                PT   
007907eb-71b5-4cfd-914b-a5a0257ab41b TORT                    FT   
                                     EU                      FT   
                                     LT2                     FT   
0093b802-ce2e-471b-be97-6493ed24114b TORT                    PT   
                                     CRIMINAL                PT   
0096e84c-70bf-4b17-928d-d451bb12283e CONAD                   FT   
                                     CONTRACT                FT   
                                     LSM                     FT   
009c01fc-be79-40e5-aa25-879731ddd8a4 CONAD                   FT   
                                     CONTRACT                FT   
                                     LSM                     FT   
                                     TORT                    FT   
00f3e3c7-43df-468f-b93e-117905d43bb3 CONTRACT                FT   
010a13b0-f40c-4cd4-9816-1c8842b4f9cf CONAD                   FT   
                                     CONTRACT                FT   
                                     LSM                     FT   
                                     TORT                    FT   
0147dbf4-0d5b-426a-a441-41ec028033d3 CONAD                   FT   
                                     CONTRACT                FT   
                                     LSM                     FT   
                                     TORT                    FT   
...                                                         ...   
fdc59029-bd0b-4599-b609-d2591f793b33 LAND                    PT   
                                     EQUITY                  PT   
                                     EU                      PT   
fdd237ba-2b86-4279-a257-b6bfd37a43f2 CONAD              Changed   
                                     CONTRACT           Changed   
                                     LSM                Changed   
fde2e280-a4ee-4f38-9164-8542098b33d7 CONTRACT                FT   
                                     LSM                     FT   
                                     TORT                    FT   
                                     CRIMINAL                FT   
fdfb1dc9-209a-4208-aa0a-0f0430784d01 EU                      PT   
                                     LT1                     PT   
                                     LT2                     PT   
fe0f4f78-96fd-4762-a299-646b19ee6f05 CONAD                   PT   
                                     LSM                     PT   
fe5f7999-6441-4ea3-bacd-dd16c5616d4d TORT                    PT   
                                     LAND                    PT   
                                     CRIMINAL                PT   
ff064eeb-a58b-486d-bc32-abb8cb97cc1e CONAD                  6YR   
                                     CONTRACT               6YR   
                                     TORT                   6YR   
ff28d841-6871-4623-8f7d-c3e063b9948c CONTRACT                FT   
                                     TORT                    FT   
                                     CRIMINAL                FT   
ff7f2ebe-64ba-460d-9194-f3fd65fd072a TORT                    PT   
                                     CRIMINAL                PT   
ffa93855-20b7-4abb-9555-5974008f5d0e TORT                    FT   
                          

In [34]:
first_attempts.loc[first_attempts['Module Result'] == 'P calc']

,SPRcode,Module,Student Programme,Student Final Status,Year of Attempt,Attempt Type,Module Mark,Module Grade,Module Result
26,007907eb-71b5-4cfd-914b-a5a0257ab41b,EU,FT,Graduated,201516,Initial Assessment,40.0,P calc,P calc
129,04010652-8cb2-42ab-920a-192dcfa7b573,EU,FT,NaN,201516,Initial Assessment,47.5,P calc,P calc
182,0530420c-8efa-4bba-b810-8266e0fc5d3d,EU,FT,Graduated,201718,Initial Assessment,46.8,P calc,P calc
209,05826d57-d01f-4e20-ba9a-bf96ebddfb0d,LAND,FT,Graduated,201516,Initial Assessment,43.5,P calc,P calc
276,074092a9-70d4-47da-a1f9-ee11c5e219f8,CONTRACT,PT,Graduated,201314,Initial Assessment,41.5,P calc,P calc
434,09e2d987-eca9-4ec4-af37-ea058a8a1073,LSM,FT,Left prematurely,201314,Initial Assessment,49.2,P calc,P calc
672,10053842-72d7-47ff-96e0-93bb56882b09,EQUITY,FT,Progressing,201718,Initial Assessment,54.0,P calc,P calc
733,124499ef-5c10-4d54-92f0-120813bfc35d,EU,FT,Graduated,201718,Initial Assessment,40.8,P calc,P calc
782,14686345-f6d6-4e3a-8629-46173616dd20,CRIMINAL,FT,Progressing,201617,Initial Assessment,45.0,P calc,P calc
783,14686345-f6d6-4e3a-8629-46173616dd20,EQUITY,FT,Progressing,201617,Initial Assessment,41.0,P calc,P calc


In [35]:
tuples = first_attempts.loc[first_attempts['Module Result'] == 'P calc', ['SPRcode', 'Module']].apply(lambda x: (x['SPRcode'], x['Module']), axis=1).tolist()

tuples


[('007907eb-71b5-4cfd-914b-a5a0257ab41b', 'EU'),
 ('04010652-8cb2-42ab-920a-192dcfa7b573', 'EU'),
 ('0530420c-8efa-4bba-b810-8266e0fc5d3d', 'EU'),
 ('05826d57-d01f-4e20-ba9a-bf96ebddfb0d', 'LAND'),
 ('074092a9-70d4-47da-a1f9-ee11c5e219f8', 'CONTRACT'),
 ('09e2d987-eca9-4ec4-af37-ea058a8a1073', 'LSM'),
 ('10053842-72d7-47ff-96e0-93bb56882b09', 'EQUITY'),
 ('124499ef-5c10-4d54-92f0-120813bfc35d', 'EU'),
 ('14686345-f6d6-4e3a-8629-46173616dd20', 'CRIMINAL'),
 ('14686345-f6d6-4e3a-8629-46173616dd20', 'EQUITY'),
 ('173218af-76af-46dd-9b77-b53a390c9de7', 'LAND'),
 ('173218af-76af-46dd-9b77-b53a390c9de7', 'EQUITY'),
 ('1739c939-086b-49e2-903f-5f3c92e542be', 'CONTRACT'),
 ('1739c939-086b-49e2-903f-5f3c92e542be', 'TORT'),
 ('1739c939-086b-49e2-903f-5f3c92e542be', 'LAND'),
 ('19229382-32b9-4ce2-95aa-32c11e3fb121', 'CONAD'),
 ('19229382-32b9-4ce2-95aa-32c11e3fb121', 'CONTRACT'),
 ('19229382-32b9-4ce2-95aa-32c11e3fb121', 'LSM'),
 ('1ba9273a-14b2-4b87-b18e-6b19d45e54e7', 'LT1'),
 ('1ba9273a-14b2-4b

In [36]:
ATTEMPTS['second'].loc[tuples, :]

,,Student Programme,Student Final Status,Year of Attempt,Attempt Type,Module Mark,Module Grade,Module Result
SPRcode,Module,,,,,,,
007907eb-71b5-4cfd-914b-a5a0257ab41b,EU,FT,Graduated,201516,FR Reassessment,NaN,NaN,F calc?
04010652-8cb2-42ab-920a-192dcfa7b573,EU,FT,NaN,201516,FR Reassessment,64.0,P,P
0530420c-8efa-4bba-b810-8266e0fc5d3d,EU,FT,Graduated,201718,DR Reassessment,75.0,P,P
05826d57-d01f-4e20-ba9a-bf96ebddfb0d,LAND,FT,Graduated,201516,DR Reassessment,61.0,P,P
074092a9-70d4-47da-a1f9-ee11c5e219f8,CONTRACT,PT,Graduated,201314,FR Reassessment,53.0,P,P
09e2d987-eca9-4ec4-af37-ea058a8a1073,LSM,FT,Left prematurely,201314,FR Reassessment,54.0,P,P
10053842-72d7-47ff-96e0-93bb56882b09,EQUITY,FT,Progressing,201718,DR Reassessment,54.0,P,P
124499ef-5c10-4d54-92f0-120813bfc35d,EU,FT,Graduated,201718,DR Reassessment,53.0,P,P
14686345-f6d6-4e3a-8629-46173616dd20,CRIMINAL,FT,Progressing,201718,Retake,45.0,DR,D


In [37]:
num_attempt_2_is_retake = len(second_attempts[second_attempts['Attempt Type'].str.contains('Retake')]) 
num_pass_attempt_2_when_retake = len(second_attempts[(second_attempts['Attempt Type'].str.contains('Retake') 
                                                & ((second_attempts['Module Result'] == 'P') 
                                                   | (second_attempts['Module Result'] == 'P calc')))])
prop_pass_attempt_2_when_retake = num_pass_attempt_2_when_retake / num_attempt_2_is_retake
prop_pass_attempt_2_when_retake

0.45734597156398105

In [38]:
num_attempt_2_is_reassess = len(second_attempts[second_attempts['Attempt Type'].str.contains('FR Reassessment')]) # 1079
num_pass_attempt_2_when_reassess = len(second_attempts[(second_attempts['Attempt Type'].str.contains('FR Reassessment')
                                                      & ((second_attempts['Module Result'] == 'P') 
                                                       | (second_attempts['Module Result'] == 'P calc')))])
prop_pass_attempt_2_when_reassess = num_pass_attempt_2_when_reassess / num_attempt_2_is_reassess
prop_pass_attempt_2_when_reassess

0.38330757341576505

In [46]:
# Make a df with each attempt pass rate as cell value, columns = years, rows = modules

############### 
'''May want to add a condition that must have failed first attempt'''
############
second_attempts = ATTEMPTS['second'].copy()
second_attempts.reset_index(inplace=True)

modules = ['LSM', 'CONTRACT', 'CONAD']#, 'TORT', 'CRIMINAL', 'LAND', 'EQUITY', 'EU', 'LT1', 'LT2']
years = sorted(second_attempts['Year of Attempt'].value_counts().index.tolist())

second_attempts_reassess_pt_yrs = pd.DataFrame()
second_attempts_reassess_ft_yrs = pd.DataFrame()
second_attempts_retake_pt_yrs = pd.DataFrame()
second_attempts_retake_ft_yrs = pd.DataFrame()


for year in years:
    for module in modules:
        pt_reassess_selection_total = ((second_attempts['Student Programme'] == 'PT') 
                           & second_attempts['Attempt Type'].str.contains('FR Reassessment') 
                           & (second_attempts['Module'] == module)
                           & (second_attempts['Year of Attempt'] == year))  
        pt_reassess_selection_passed = (pt_reassess_selection_total & ((second_attempts['Module Result'] == 'P') 
                                              | (second_attempts['Module Result'] == 'P calc'))) 
        pt_retake_selection_total = ((second_attempts['Student Programme'] == 'PT') 
                           & second_attempts['Attempt Type'].str.contains('Retake') 
                           & (second_attempts['Module'] == module)
                           & (second_attempts['Year of Attempt'] == year)) 
        pt_retake_selection_passed = (pt_retake_selection_total & ((second_attempts['Module Result'] == 'P') 
                                              | (second_attempts['Module Result'] == 'P calc')))
        
        ft_reassess_selection_total = ((second_attempts['Student Programme'] == 'FT') 
                           & second_attempts['Attempt Type'].str.contains('FR Reassessment') 
                           & (second_attempts['Module'] == module)
                           & (second_attempts['Year of Attempt'] == year))  
        ft_reassess_selection_passed = (ft_reassess_selection_total & ((second_attempts['Module Result'] == 'P') 
                                              | (second_attempts['Module Result'] == 'P calc'))) 
        ft_retake_selection_total = ((second_attempts['Student Programme'] == 'FT') 
                           & second_attempts['Attempt Type'].str.contains('Retake') 
                           & (second_attempts['Module'] == module)
                           & (second_attempts['Year of Attempt'] == year)) 
        ft_retake_selection_passed = (ft_retake_selection_total & ((second_attempts['Module Result'] == 'P') 
                                              | (second_attempts['Module Result'] == 'P calc')))
        
        if (int(len(second_attempts[pt_reassess_selection_total])) != 0):
            second_attempts_reassess_pt_yrs.at[module, year] = \
            len(second_attempts[pt_reassess_selection_passed])/len(second_attempts[pt_reassess_selection_total]) 
        
        if (int(len(second_attempts[pt_retake_selection_total])) != 0):
            second_attempts_retake_pt_yrs.at[module, year] = \
            len(second_attempts[pt_retake_selection_passed])/len(second_attempts[pt_retake_selection_total])
            
        if (int(len(second_attempts[ft_reassess_selection_total])) != 0):
            second_attempts_reassess_ft_yrs.at[module, year] = \
            len(second_attempts[ft_reassess_selection_passed])/len(second_attempts[ft_reassess_selection_total]) 
        
        if (int(len(second_attempts[ft_retake_selection_total])) != 0):
            second_attempts_retake_ft_yrs.at[module, year] = \
            len(second_attempts[ft_retake_selection_passed])/len(second_attempts[ft_retake_selection_total])            

In [47]:
second_attempts_reassess_pt_yrs.mean()

201112    0.459596
201213    0.363426
201314    0.272487
201415    0.113112
201516    0.189001
201617    0.491453
201718    0.246693
dtype: float64

In [45]:
second_attempts_retake_pt_yrs.mean()

201213    0.037037
201314    0.399564
201415    0.400000
201516    0.266667
201617    0.333333
201718    0.278571
dtype: float64

In [42]:
second_attempts_reassess_ft_yrs

,201112,201213,201314,201415,201516,201617,201718
LSM,0.600000,0.857143,0.666667,0.308642,0.075000,0.538462,0.629630
CONTRACT,0.375000,0.526316,0.342857,0.191489,0.229508,0.214286,0.600000
CONAD,0.545455,0.529412,0.250000,0.203704,0.090909,0.500000,0.090909
TORT,0.500000,0.500000,0.219178,0.155172,0.212766,0.272727,0.409091
LAND,0.500000,0.916667,0.705882,0.250000,0.363636,0.375000,0.071429
CRIMINAL,NaN,0.750000,0.363636,0.363636,0.666667,0.666667,0.000000
EQUITY,NaN,1.000000,0.400000,0.476190,0.538462,0.200000,0.000000
EU,NaN,NaN,0.333333,0.700000,0.461538,0.800000,0.600000
LT1,NaN,NaN,0.500000,0.666667,0.142857,NaN,0.500000
LT2,NaN,NaN,1.000000,0.750000,0.230769,NaN,0.800000


In [43]:
second_attempts_retake_ft_yrs

,201314,201415,201516,201617,201718
LSM,0.000000,0.5,0.0,0.000000,0.333333
CONTRACT,0.285714,0.0,0.0,0.000000,0.384615
CONAD,0.000000,0.0,0.0,0.111111,0.388889
TORT,0.200000,0.0,0.0,0.230769,0.400000
CRIMINAL,0.400000,1.0,1.0,NaN,0.333333
LAND,0.933333,0.0,NaN,0.000000,0.750000
EQUITY,NaN,NaN,NaN,0.166667,0.400000
EU,NaN,NaN,NaN,0.000000,0.666667
LT1,NaN,NaN,NaN,0.000000,0.625000
LT2,NaN,NaN,NaN,0.000000,0.571429


In [52]:
# Make a df with each attempt pass rate as cell value, columns = years, rows = modules

modules = ['LSM', 'CONTRACT', 'CONAD']#, 'TORT', 'CRIMINAL', 'LAND', 'EQUITY', 'EU', 'LT1', 'LT2']
years = sorted(first_attempts['Year of Attempt'].value_counts().index.tolist())

first_attempts_pt_yrs = pd.DataFrame()
first_attempts_ft_yrs = pd.DataFrame()

pt_selection_total = pd.DataFrame()
ft_selection_total = pd.DataFrame()

for year in years:
    for module in modules:
        pt_selection_total[year] = ((first_attempts['Student Programme'] == 'PT')  
                           & (first_attempts['Module'] == module)
                           & (first_attempts['Year of Attempt'] == year))  
        pt_selection_passed = (pt_selection_total[year] & ((first_attempts['Module Result'] == 'P') 
                                              | (first_attempts['Module Result'] == 'P calc')))    
        ft_selection_total[year] = ((first_attempts['Student Programme'] == 'FT') 
                           & (first_attempts['Module'] == module)
                           & (first_attempts['Year of Attempt'] == year))  
        ft_selection_passed = (ft_selection_total[year] & ((first_attempts['Module Result'] == 'P') 
                                              | (first_attempts['Module Result'] == 'P calc'))) 
           
        if (int(len(first_attempts[pt_selection_total[year]])) != 0):
            first_attempts_pt_yrs.at[module, year] = \
            len(first_attempts[pt_selection_passed])/len(first_attempts[pt_selection_total[year]]) 
            
        if (int(len(first_attempts[ft_selection_total[year]])) != 0):
            first_attempts_ft_yrs.at[module, year] = \
            len(first_attempts[ft_selection_passed])/len(first_attempts[ft_selection_total[year]])            

In [50]:
first_attempts_pt_yrs.mean()

201112    0.687658
201213    0.591317
201314    0.471270
201415    0.510411
201516    0.479630
201617    0.502956
201718    0.496457
dtype: float64

In [51]:
first_attempts_ft_yrs.mean()

201112    0.795971
201213    0.609933
201314    0.576375
201415    0.510627
201516    0.543792
201617    0.535669
201718    0.578153
dtype: float64

In [59]:
first_attempts[pt_selection_total].sum(axis=0)

SPRcode                 0.0
Module                  0.0
Student Programme       0.0
Student Final Status    0.0
Year of Attempt         0.0
Attempt Type            0.0
Module Mark             0.0
Module Grade            0.0
Module Result           0.0
dtype: float64

In [ ]:
first_attempts_ft_yrs.iloc[0:4].mean()

In [ ]:
first_attempts_ft_yrs.mean(axis=1)

In [ ]:
first_attempts_pt_yrs.iloc[0:3].mean()

In [ ]:
first_attempts_pt_yrs.mean(axis=1)

In [ ]:
third_attempts = ATTEMPTS['third']

In [ ]:
third_attempts['Attempt Type'].value_counts(dropna=False)

In [ ]:
ATTEMPTS['third']['Module Result'].value_counts(dropna=False)

In [ ]:
ATTEMPTS['fourth']

In [ ]:
ATTEMPTS['fifth']

In [ ]:
third_attempts[third_attempts['Attempt Type'].isnull()]